In [136]:
import pandas as pd
import conllu as cl
from os.path import join
import math

In [137]:
class Quad:
    def __init__(self, entity, indexSentence, indexStart, indexEnd):
        self.entity = entity
        self.indexSentence = indexSentence
        self.indexStart = indexStart
        self.indexEnd = indexEnd
    
    def __eq__(self, other):
        return (self.entity == other.entity and
                self.indexSentence == other.indexSentence and
                self.indexStart == other.indexStart and
                self.indexEnd == other.indexEnd)
    
    def __hash__(self):
        return hash((self.entity, self.indexSentence, self.indexStart, self.indexEnd))

    def toString(self):
        return f"Quad(Entity={self.entity}, indexSentence={self.indexSentence}, indexStart={self.indexStart}, indexEnd={self.indexEnd})"

In [138]:
def reconstructSentences(sentences):
    modified_sentences = []
    for sentence in sentences:
        modified_sentence = []
        # create a start token
        new_token_start = {
            "id": len(sentence),
            "form": "<s>",
            "lemma": "START",
        }
        # create a end token
        new_token_end = {
            "id": len(sentence) + 1,
            "form": "</s>",
            "lemma": "END",
        }
        # append the start token
        modified_sentence.append(new_token_start)
        # append the rest of the sentence
        for word in sentence:
            modified_sentence.append(word)
        # append the end token
        modified_sentence.append(new_token_end)

        # append the new sentence to the list of sentences
        modified_sentences.append(modified_sentence)

    return modified_sentences

In [139]:
def load(language):
    filePath = ".\Datasets\\" + language + "\\"
    
    with open(join(filePath, "train.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    train_sentences = reconstructSentences(cl.parse(data))

    with open(join(filePath, "test.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    test_sentences = cl.parse(data)

    with open(join(filePath, "val.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    val_sentences = cl.parse(data)

    return train_sentences, test_sentences, val_sentences

In [140]:
# Fit the HMM model with the train sentences, it generates 2 outputps containing the probabilities that will be used 
# in the decoding phase.
def HMMFit(train_sentences):
    # dictionary of dictionaries
    countTag = {}   # Count(tag(i))
    transitionCount = {}    # Count(tag(i-1), tag(i))
    wordTagCount = {}   # Count(tag(i), word(i))
    tagDistribution = {}    # Count(tag(i-1), tag(i)) / Count(tag(i-1))
    wordDistribution = {}   # Count(tag(i), word(i))

    # complete creation of counTag dictorary
    # partial initialization of transitionCount dictionary
    # partial creation of wordTagCount dictionary
    for sentence in train_sentences:
        for word in sentence:
            if word['lemma'] in countTag:
                countTag[word['lemma']] += 1
            else:
                countTag[word['lemma']] = 1
                transitionCount[word['lemma']] = {}
            
            if word['form'] not in wordTagCount:
                wordTagCount[word['form']] = {}
            if word['lemma'] in wordTagCount[word['form']]:
                wordTagCount[word['form']][word['lemma']] += 1
            else:
                wordTagCount[word['form']][word['lemma']] = 1
    
    # complete inizialization of transitionCount dictionary
    for key in transitionCount.keys():
        for tag in countTag.keys():
            transitionCount[key][tag] = 0

    # complete creation of wordTagCount dictionary
    for word in wordTagCount.keys():
        for tag in countTag.keys():
            if tag not in wordTagCount[word]:
                wordTagCount[word][tag] = 0

    # complete creation of transitionCount dictionary
    # previousTag --> Markov's hypothesis of first grade
    for tag in transitionCount.keys():
        for sentence in train_sentences:
            previousTag = None
            for word in sentence:
                if previousTag == None:
                    previousTag = word['lemma']
                else:
                    if word['lemma'] == tag:
                        if previousTag in transitionCount[tag]:
                            transitionCount[tag][previousTag] += 1
                    previousTag = word['lemma']
    
    # complete creation of tagDistribution dictionary
    # we use the log-probability for better accuracy (since the probability are very small)
    # if the probability is 0, then the log-probability will be log(10**-10) 
    epsilon = 10**-10
    for eventTag in countTag.keys():
        tagDistribution[eventTag] = {}
        for condTag in countTag.keys():
            if transitionCount[eventTag][condTag]/countTag[condTag] == 0:
                tagDistribution[eventTag][condTag] = math.log(epsilon)
            else:
                tagDistribution[eventTag][condTag] = math.log(transitionCount[eventTag][condTag]/countTag[condTag])
            
    # complete creation of wordDistribution dictionary
    # we use the log-probability for better accuracy (since the probability are very small)
    # if the probability is 0, then the log-probability will be log(10**-10)
    for word in wordTagCount.keys():
        wordDistribution[word] = {}
        for tag in countTag.keys():
            if wordTagCount[word][tag]/countTag[tag] == 0:
                wordDistribution[word][tag] = math.log(epsilon)
            else:
                wordDistribution[word][tag] = math.log(wordTagCount[word][tag]/countTag[tag])
        
    return tagDistribution, wordDistribution

In [141]:
# strategy can be: 
# 0: always "O": P(unk|O) = 1 
# 1: always "O" and "MISC": P(unk|O) = P(unk|B-MISC) = 0.5
# 2: uniform: P(unk|tag(i)) = 1 / #(NER_TAGs)
def smoothing(strategy, currentTag, totNERtags):
    epsilon = 10**-10
    match(strategy):
        case 0:
            if currentTag == "O":
                return math.log(1)
            else:
                return math.log(epsilon)
        case 1: 
            if currentTag == "O" or currentTag == "B-MISC":
                return math.log(0.5)
            else:
                return math.log(epsilon)
        case 2:
            return math.log(1/totNERtags)
        case _:
            return math.log(epsilon)

In [142]:
def viterbiAlgorithm(sentence, tagDistribution, wordDistribution, b_value):
    epsilon = 10**-10
    columns = len(sentence) # words
    rows = len(tagDistribution.keys())-2 #tags
    viterbi = [[math.log(epsilon) for _ in range(columns)] for _ in range(rows)]
    backpointer = [["" for _ in range(columns)] for _ in range(rows)]

    # initialization step
    i = 0
    for tag in tagDistribution.keys():
        if tag != "START" and tag != "END": 
            viterbi[i][0] = tagDistribution[tag]["START"] + b_value
            backpointer[i][0] = "0"
            i += 1

    tagList = list(tagDistribution)
    tagList.remove("START")
    tagList.remove("END")

    # recursion step
    for i in range(1, len(sentence)):
        j = 0
        for tag in tagDistribution.keys():
            if tag != "START" and tag != "END":
                if (sentence[i])['form'] not in wordDistribution:
                    obsLikelihood = smoothing(2, tag, len(tagList))
                else:
                    obsLikelihood = wordDistribution[(sentence[i])['form']][tag]
                
                viterbi[j][i] = max(viterbi[tagLoop][i-1] + obsLikelihood + tagDistribution[tag][tagList[tagLoop]] for tagLoop in range(len(tagList)))
                _, maxTag = max((viterbi[tagLoop][i-1] + tagDistribution[tag][tagList[tagLoop]], tagLoop) for tagLoop in range(len(tagList)))
                backpointer[j][i] = maxTag
                j += 1

    # termitation step
    viterbi_end = max(viterbi[tagLoop][len(sentence)-1] + tagDistribution["END"][tagList[tagLoop]] for tagLoop in range(len(tagList)))
    _, maxTag = max((viterbi[tagLoop][len(sentence)-1] + tagDistribution["END"][tagList[tagLoop]], tagLoop) for tagLoop in range(len(tagList)))
    backpointer_end = maxTag
    
    # reconstructing the solution
    path = [None] * len(sentence)
    solution = [None] * len(sentence)
    path[len(sentence)-1] = backpointer_end
    solution[len(sentence)-1] = tagList[backpointer_end]
    for i in range(len(sentence)-2, -1, -1): # decrementing loop, 0 inclusive
        path[i] = backpointer[path[i+1]][i+1]
        solution[i] = tagList[path[i]]

    return solution

In [143]:
def convertToListEntites(predict, test_sentences):
    listTestSentences = []
    currentTag = ""
    indexSentence = 0
    indexStart = 0
    indexEnd = 0
    for sentence in test_sentences:
        for word in sentence:
            if word['lemma'] != "O": 
                lemma = word['lemma'].split("-")[1]
                if currentTag != lemma:
                    if currentTag == "O":
                        currentTag = lemma
                        indexStart = word['id']
                    elif currentTag != "":
                        indexEnd = word['id']-1
                        listTestSentences.append(Quad(currentTag, indexSentence, indexStart, indexEnd))
                    currentTag = lemma
                    indexStart = word['id']
            elif word['lemma'] == "O" and currentTag != "" and currentTag != "O":
                indexEnd = word['id']-1
                listTestSentences.append(Quad(currentTag, indexSentence, indexStart, indexEnd))
                currentTag = "O"

        indexSentence += 1

    listPredict = []
    currentTag = ""
    indexSentence = 0
    indexStart = 0
    indexEnd = 0
    for i in range(len(predict)):
        for j in range(len(predict[i])):
            if predict[i][j] != "O":
                lemma = predict[i][j].split("-")[1]
                if currentTag != lemma:
                    if currentTag == "O":
                        currentTag = lemma
                        indexStart = j
                    elif currentTag != "":
                        indexEnd = j-1
                        listPredict.append(Quad(currentTag, i, indexStart, indexEnd))
                    currentTag = lemma
                    indexStart = j
            elif predict[i][j] == "O" and currentTag != "" and currentTag != "O":
                indexEnd = j-1
                listPredict.append(Quad(currentTag, i, indexStart, indexEnd))
                currentTag = "O"
    
    return listPredict, listTestSentences


In [144]:
def convertListToDict(list):
    dict = {}
    for quad in list:
        key = (quad.entity, quad.indexSentence, quad.indexStart, quad.indexEnd)
        dict[key] = quad
    
    return dict

In [145]:
def precisionAndRecall(predictions, test_sentences, trueClass):

    dict_predictions = convertListToDict(predictions)
    dict_testSentences = convertListToDict(test_sentences)    

    truePositive = [pred for pred in predictions if pred.entity == trueClass and (pred.entity, pred.indexSentence, pred.indexStart, pred.indexEnd) in dict_testSentences]
    falsePositive = [pred for pred in predictions if pred.entity == trueClass and (pred.entity, pred.indexSentence, pred.indexStart, pred.indexEnd) not in dict_testSentences]
    falseNegative = [test for test in test_sentences if test.entity == trueClass and (test.entity, test.indexSentence, test.indexStart, test.indexEnd) not in dict_predictions]

    precision = len(truePositive) / (len(truePositive) + len(falsePositive)) if (len(truePositive) + len(falsePositive)) > 0 else 0
    recall = len(truePositive) / (len(truePositive) + len(falseNegative)) if (len(truePositive) + len(falseNegative)) > 0 else 0

    return precision, recall        

In [167]:
def evaluation(predict, test_sentences, tagDistribution):
    predictions = 0
    correctPredictions = 0
    
    tagList = list(tagDistribution)
    tagList.remove("START")
    tagList.remove("END")
    tagList.remove("O")
    for i in range(len(tagList)):
        tagList[i] = tagList[i].split("-")[1]
    entity = list(set(tagList))

    # accuracy
    i = 0
    for sentence in test_sentences:
        j = 0
        for word in sentence:
            if word['lemma'] == predict[i][j]:
                correctPredictions += 1
            predictions += 1
            j += 1
        i += 1
    accuracy = correctPredictions / predictions

    listPredict, listTestSentences = convertToListEntites(predict, test_sentences)
    
    dictPrecision = {}
    dictRecall = {}
    for i in range(len(entity)):
        precision, recall = precisionAndRecall(listPredict, listTestSentences, entity[i])
        dictPrecision[entity[i]] = precision
        dictRecall[entity[i]] = recall

    return accuracy, dictPrecision, dictRecall

In [147]:
def HMMPredict(test_senteces, tagDistribution, wordDistribution):
    b_value = 0 # in log probabilities 0 is the neutral element
    solution = []

    for sentence in test_senteces:
        sentenceSolution = viterbiAlgorithm(sentence, tagDistribution, wordDistribution, b_value)
        # recontruct and add to the solution...
        solution.append(sentenceSolution)

    return solution

In [148]:
it_train_sentences, it_test_sentences, it_val_sentences = load("it")
#en_train_sentences, en_test_sentences, en_val_sentences = load("en")
#es_train_sentences, es_test_sentences, es_val_sentences = load("es")

In [149]:
tagDistribution, wordDistribution = HMMFit(it_train_sentences)

In [150]:
predict = HMMPredict(it_test_sentences, tagDistribution, wordDistribution)

In [169]:
accuracy, precision, recall = evaluation(predict, it_test_sentences, tagDistribution)

precisionTot = 0
recallTot = 0
print("The model's accuracy is: " +  str(accuracy))
print()
print("Entities' precision and recall:")
for key in precision.keys():
    print("Precision " + key + ": " + str(precision[key]))
    print("Recall " + key + ": " + str(recall[key]))
    precisionTot = precisionTot + precision[key]
    recallTot = recallTot + recall[key]
    print()

precisionTot = precisionTot / len(precision.keys())
recallTot = recallTot / len(recall.keys())
print("Average precision: " + str(precisionTot))
print("Average recall: " + str(precisionTot))

The model's accuracy is: 0.9740821424236423

Entities' precision and recall:
Precision MISC: 0.6895287958115183
Recall MISC: 0.5580508474576271

Precision PER: 0.9137792103142627
Recall PER: 0.8125149271554812

Precision LOC: 0.8690170701773959
Recall LOC: 0.7973180468829972

Precision ORG: 0.8438250840941854
Recall ORG: 0.789923526765632

Average precision: 0.8290375400993406
Average recall: 0.8290375400993406
